# KDD 2009 - Tratamento de dados simplificado

Este notebook foi criado para servir de exemplo de implementação de um modelo preditivo. O modelo mostra parte da solução do [KDD Cup de 2009](https://www.kdd.org/kdd-cup/view/kdd-cup-2009/Tasks).

## Descrição da tarefa do KDD 2009
A tarefa consiste em estimar as probabilidades de churn, appetency e up-selling de clisntes, já que estes são os tres objetivos a prever. Há duas variações do desafio, uma maior com 15.000 variáveis e outra, menor, com 230 variáveis.

Definições:
- ***Churn***: A taxa de *churn*, também chamada de taxa de atrito, é um dos dois fatores primários que determinam a estabilidade de um cliente que o negócio irá suportar. Em um sentido mais amplo, a taxa de *churn* é a medida do número de indivíduos ou itens que se movem para e de uma coleção em um período específico de tempo. O termo é utilizado em vários contextos, mas é aplicado de forma mais ampla em negócios no que tange a base contratual de clientes.
- ***Appetency***: No nosso contexto, *appetency*, é a propensão para comprar um serviço ou produto.
- ***Up-selling***: *Up-selling* é um técnica de venda onde um vendedor faz com que um consumidor compre itens mais caros, com upgrades, ou produtos adicionais em uma tentativa de aumentar o lucro das vendas. *Up-selling* normalmente indica vender produtos adicionais, que devem ser mais lucrativos que a venda original.


## Abordagem
Para resolver o problema baseado nos dados mais simples (230 variáveis do [KDD 2009](https://www.kdd.org/kdd-cup/view/kdd-cup-2009/Tasks)) seguiremos o seguinte roteiro geral:

1. Ler os dados do KDD 2009
2. Tratar os dados de forma simples, para extrair variáveis vazias ou com todos os valores nulos, que não tem impacto no modelo preditivo
3. Preparar os dados para utilizar o modelo preditivo escolhido (ver item 5)
4. Separar os dados em tres grupos:
    - Treinamento
    - Testes (ou validação)
    - Validação cruzada
5. Treinar o modelo com [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html), escolhendo o número de rodadas de execução que produza o melhor resultado no grupo de testes. A métrica utilizada para avalizar o modelo, como descrito no desafio, será [AUC](https://en.wikipedia.org/wiki/AUC)
6. Validar este resultado com o grupo de validação cruzada. O resultado final será o mínimo entre os dois resultados
7. Como exercício adicional, vamos comparar a curva de resultados entre Testes (Validação) e Validação Cruzada, para avalizar, de forma superficial, como o modelo se comporta e verificar se há tendência de *overfitting* ou *underfitting*

In [1]:
# Importação de bibliotecas
import math
import numpy as np
import os
import pandas as pd
import pickle
import random
from sklearn.metrics import auc
import sys
import xgboost as xgb

# Rotinas de visualização de dados
from bokeh.io import output_notebook, push_notebook, show, export_png
from bokeh.models import Range1d
from bokeh.plotting import figure

output_notebook()

Loading BokehJS ...

# Lendo arquivo de treinamento

O arquivo com dados de treinamento possui 50000 entradas. Os arquivos com os rótulos (churn, appetency, upselling) estão em separado e serão lidos mais tarde.

Os arquivos estão em formato TSV (valores separados por tab, com valores "não disponíveis" em branco

Por exemplo (os espaços são apenas ilustrativos)
```
Var1	Var2	Var3	Var4	Var5	Var6	Var7	Var8	Var9	Var10
\t      \t      \t      \t      1024\t    \t      \t    2048\t  \t      \t
```

Para ler estes dados usamos a função do pandas [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) indicando que o delimitador entre campos é o caracter tab (*\t*) e que os valores em branco/nulos são vazios.

Os valores são carregados no [dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) `kdd2009`.

In [2]:
# Diretório de dados
data_folder = os.path.join("..", "data")
# Arquivo de dados
training_data_file = os.path.join(data_folder, "orange_small_train.data")

# Dataframe com os dados de treinamento
kdd2009 = pd.read_csv(training_data_file, delimiter='\t', na_values=[''])

## Limpeza dos dados
Uma vez lidos os dados, é necessário verificar se há colunas apenas com NaNs (not a number) e remover estas colunas, caso existam. 

Para auxiliar o processo, criamos um dataframe com a contagem de NaNs (Not a Number) de cada coluna dos dados da Orange e depois criamos a lista das colunas que só possuem NaNs \([Not a Number](https://docs.python.org/3/library/math.html?highlight=nan#math.nan)\), mais tarde essas colunas serão removidas do dataframe de dados da Orange.

In [3]:
# Número total de pontos de dados
numeroDados = len(kdd2009)

# Cria dataframe com contagem de NaNs
nans = pd.DataFrame(kdd2009.isna().sum(), columns=['NAN_count'])

Examinando os 10 primeiros resultados... (por ordem de nome "***feature***")

In [4]:
nans.head(10)

,NAN_count
Var1,49298
Var2,48759
Var3,48760
Var4,48421
Var5,48513
Var6,5529
Var7,5539
Var8,50000
Var9,49298
Var10,48513


Examinando os 20 "***features***" com maior ocorrência de *NaNs*

In [5]:
nans.sort_values(by=['NAN_count'], ascending=False).head(20)

,NAN_count
Var230,50000
Var175,50000
Var55,50000
Var32,50000
Var141,50000
Var20,50000
Var39,50000
Var42,50000
Var167,50000
Var79,50000


In [6]:
# Recupera lista de variaveis que só possuem NaN
variaveisNulas = nans[nans['NAN_count'] == numeroDados].index.values

# Remove as colunas que só contém NaNs
kdd2009.drop(variaveisNulas, axis=1, inplace=True)

# Lendo arquivos de labels
Uma vez lidos os dados, é necessário ler os arquivos com os rótulos de cada um dos atributos que desejamos prever:
- *Appetency*: Propensão de contratar serviços
- *Churn*: Tendência de cancelar o contrato, também chamada de taxa de atrito)
- *Upselling*: Propensão de contratar serviços mais caros

Os rótulos estão em arquivos separados. Colunas com os tres rótulos serão incluídas no dataframe do kdd2009. Os arquivos de rótulos estão codificados em colunas, com -1 para falso e 1 para verdadeiro. Como no exemplo abaixo
```
-1
1
-1
-1
-1
-1
-1
-1
-1
-1

```

In [7]:
# Lendo arquivos com os rótulos das métricas
# Colunas de resultados
resultados = ['appetency', 'churn', 'upselling']
padraoArquivosRotulos = "orange_small_train_{}.labels"

# Colunas de variáveis de entrada
variaveis = kdd2009.keys()

# Le os arquivos de rótulos, converte 1 -> True, -1 -> False e inclui no dataframe do kdd
for rotulo in resultados:
    # Gera nome do arquivo
    nomeArquivo = os.path.join(data_folder, padraoArquivosRotulos.format(rotulo))
    # Lê arquivo
    dfRotulo = pd.read_csv(nomeArquivo, header=None, names=['label'])
    # Converte 1 -> True, -1 -> False - Assume que != 1 -> False
    dfRotulo['label'] = dfRotulo['label'] == 1
    # Inclusão dos rótulos no dataframe do kdd como colunas adicionais
    kdd2009[rotulo] = dfRotulo['label']

# Verificando os rótulos no DataFrame
Rápida visualização dos dados dos rótulos

In [8]:
# Primeiros valores dos resultados no dataframe de treinamento
kdd2009[resultados].head(10)

,appetency,churn,upselling
0,False,False,False
1,False,True,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
6,False,False,False
7,False,False,False
8,False,False,False
9,False,False,False


## E verificando as variáveis (já sem colunas de NaNs)

In [9]:
# Primeiros valores das variaveis no dataframe de treinamento
kdd2009[variaveis].head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,L91KIiz,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86


## Verificando a correlação entre colunas
Utilizando coeficiente de correlação de [Pearson](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)

In [10]:
correlacao = kdd2009[variaveis].corr(method='pearson')
correlacao.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
Var1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.900828,NaN,NaN,...,0.210084,NaN,NaN,NaN,NaN,0.000525,0.880618,NaN,-0.068517,NaN
Var2,NaN,1.000000,-0.002711,NaN,NaN,NaN,NaN,NaN,NaN,-0.006198,...,NaN,NaN,NaN,0.030137,0.050524,NaN,NaN,0.018761,-0.068326,NaN
Var3,NaN,-0.002711,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.003885,...,NaN,NaN,NaN,0.081396,0.021622,NaN,NaN,0.093520,0.009123,NaN
Var4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.03495,NaN,NaN,NaN,NaN,NaN,0.026948,NaN
Var5,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.161563,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.071298,0.137581


## Lista variáveis com maior índice de correlação
Para cada uma das variáveis, lista as que possuem o mais alto índice de correlação

In [11]:
limite = 0.99

In [12]:
for variavel in correlacao.index:
    correlacoes = correlacao[correlacao[variavel].abs() > limite]
    if len(correlacoes) > 1:
        print('---> Coluna: {}'.format(variavel))
        for coluna in correlacoes.index:
            if coluna != variavel:
                print('{}: {}'.format(coluna, correlacoes[variavel][coluna]))
        print()

---> Coluna: Var9
Var66: 0.9999996441554745
Var156: 0.9999996441554707

---> Coluna: Var21
Var22: 0.9999999999998949

---> Coluna: Var22
Var21: 0.9999999999998949

---> Coluna: Var66
Var9: 0.9999996441554745
Var156: 1.0000000000000075

---> Coluna: Var71
Var91: 0.9999999999999943
Var148: 0.9999999999999943

---> Coluna: Var88
Var128: 0.9999999999999976
Var145: 1.0000000000000038

---> Coluna: Var91
Var71: 0.9999999999999943
Var148: 1.0

---> Coluna: Var104
Var105: 0.9999999999999978

---> Coluna: Var105
Var104: 0.9999999999999978

---> Coluna: Var128
Var88: 0.9999999999999976
Var145: 1.000000000000007

---> Coluna: Var145
Var88: 1.0000000000000038
Var128: 1.000000000000007

---> Coluna: Var148
Var71: 0.9999999999999943
Var91: 1.0

---> Coluna: Var156
Var9: 0.9999996441554707
Var66: 1.0000000000000075



# Conversão de colunas de objetos e booleanos para inteiros

### Modelo baseado no XGBoost
Para utilizar o [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html) é necessário converter as colunas de objetos e valores booleanos para inteiros ou reais.

Inicialmente vamos classificar as colunas em inteiros, reais, objetos e booleanos e depois converter objetos e booleanos para inteiros, já que o algoritmo que será utilizado para treinamento do modelo requer que os dados sejam inteiros ou reais.

In [13]:
# Verifica os tipos de dados existentes no dataframe
print('Lista de tipos de dados no kdd2009: {}'.format(list(set(kdd2009[variaveis].dtypes))))

Lista de tipos de dados no kdd2009: [dtype('O'), dtype('int64'), dtype('float64')]


In [14]:
data_types = pd.DataFrame(kdd2009[variaveis].dtypes, columns=['dtype'])

variaveisInteiras = list(data_types[data_types.values == np.dtype('int64')].index.values)
variaveisReais = list(data_types[data_types.values == np.dtype('float64')].index.values)
variaveisCategoricas = list(data_types[data_types.values == np.dtype('object')].index.values)

len(variaveisInteiras+variaveisReais+variaveisCategoricas+resultados) == len(kdd2009.keys())

print('Há {} variáveis inteiras, {} variáveis reais e {} objetos'.format(
                                                                    len(variaveisInteiras),
                                                                    len(variaveisReais),
                                                                    len(variaveisCategoricas)))

# Verifica se o total de tipos de variáveis está correto
if len(variaveisInteiras+variaveisReais+variaveisCategoricas+resultados) != len(kdd2009.keys()):
    print('Aviso: erro na contagem de tipos de variáveis!!!')

Há 1 variáveis inteiras, 173 variáveis reais e 38 objetos


## Breve verificação dos dados de cada tipo

In [15]:
kdd2009[variaveisCategoricas].head()

,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,NaN,bZkvyxLkBI,RO12,NaN,taul,1K8T,lK27,ka_ns41,nQUveAzAF7,NaN,...,1YVfGrO,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN
1,NaN,CEat0G8rTN,RO12,NaN,taul,1K8T,2Ix5,qEdASpP,y2LIM01bE1,NaN,...,0AJo2f2,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN
2,NaN,eOQt0GoOh3,AERks4l,SEuy,taul,1K8T,ffXs,NldASpP,y4g9XoZ,vynJTq9,...,JFM1BiF,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c
3,NaN,jg69tYsGvO,RO12,NaN,taul,1K8T,ssAy,_ybO0dd,4hMlgkf58mhwh,NaN,...,L91KIiz,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN
4,NaN,IXSgUHShse,RO12,SEuy,taul,1K8T,uNkU,EKR938I,ThrHXVS,0v21jmy,...,OrnLfvc,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86


## Cálculos de estatísticas descritivas básicas das colunas de dados

In [16]:
kdd2009.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var180,Var181,Var182,Var183,Var184,Var186,Var187,Var188,Var189,Var190
count,702.000000,1241.000000,1240.000000,1579.000000,1.487000e+03,44471.000000,44461.000000,702.000000,1.487000e+03,1240.000000,...,7.020000e+02,44991.000000,1.579000e+03,1.241000e+03,1241.000000,702.000000,702.00000,1241.000000,21022.000000,333.000000
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,48.145299,3.926057e+05,8.625806,...,3.776755e+06,0.611456,1.416638e+06,7.777380e+04,8.460919,3.299145,16.54416,167.368477,270.142137,22007.045192
std,40.709951,0.141933,4270.193518,1.275481,6.441259e+05,2685.693668,6.326053,154.777855,9.280896e+05,2.869558,...,3.785696e+06,2.495681,2.279786e+06,2.016188e+05,46.973777,8.781967,60.22303,113.980072,86.707692,29085.146490
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,8.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,-6.420000,6.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,518.000000,0.000000,4.000000,0.000000e+00,8.000000,...,1.917352e+05,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.00000,19.380000,204.000000,2732.670000
50%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,861.000000,7.000000,20.000000,0.000000e+00,8.000000,...,2.431310e+06,0.000000,1.167780e+05,0.000000e+00,0.000000,0.000000,4.00000,197.640000,270.000000,12668.940000
75%,16.000000,0.000000,0.000000,0.000000,1.187425e+05,1428.000000,7.000000,46.000000,2.628630e+05,8.000000,...,6.471827e+06,0.000000,1.844952e+06,4.881000e+04,8.000000,6.000000,14.00000,252.960000,330.000000,29396.340000
max,680.000000,5.000000,130668.000000,27.000000,6.048550e+06,131761.000000,140.000000,2300.000000,1.232559e+07,40.000000,...,1.428483e+07,49.000000,1.199478e+07,3.048400e+06,1200.000000,102.000000,910.00000,628.620000,642.000000,230427.000000


In [17]:
kdd2009.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,False,False,False
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,False,True,False
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,False,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,False,False,False
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,False,False,False


## Convertendo variáveis categóricas para inteiros (para usar no XGBoost)

In [18]:
# Cada variável categórica é convertida para Categorical do Pandas e depois
# mudamos seu valor categórico para o código numérico correspondente
for variavel in variaveisCategoricas:
    kdd2009[variavel] = pd.Categorical(kdd2009[variavel])
    kdd2009[variavel] = kdd2009[variavel].cat.codes

In [19]:
# Verifica os tipos de dados existentes no dataframe
print('Lista de tipos de dados no kdd2009: {}'.format(list(set(kdd2009[variaveis].dtypes))))

Lista de tipos de dados no kdd2009: [dtype('int16'), dtype('int8'), dtype('int64'), dtype('float64')]


In [20]:
kdd2009.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,3,-1,-1,22,2,8,-1,False,False,False
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,0,-1,-1,14,2,8,-1,False,True,False
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,3,-1,1,10,0,25,0,False,False,False
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,0,-1,-1,7,2,8,-1,False,False,False
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,0,-1,1,7,2,8,1,False,False,False


In [21]:
kdd2009.describe()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
count,702.000000,1241.000000,1240.000000,1579.000000,1.487000e+03,44471.000000,44461.000000,702.000000,1.487000e+03,1240.000000,...,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,11.487179,0.004029,425.298387,0.125396,2.387933e+05,1326.437116,6.809496,48.145299,3.926057e+05,8.625806,...,2003.12184,3.984080,2123.702060,0.302480,-0.983600,-0.220100,10.115720,2.103060,10.233400,-0.368800
std,40.709951,0.141933,4270.193518,1.275481,6.441259e+05,2685.693668,6.326053,154.777855,9.280896e+05,2.869558,...,1248.04682,1.119166,1183.541455,1.070238,0.127009,0.933506,5.798438,0.867994,6.488254,0.799074
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,8.000000,...,0.00000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,518.000000,0.000000,4.000000,0.000000e+00,8.000000,...,826.00000,4.000000,1066.000000,0.000000,-1.000000,-1.000000,6.000000,2.000000,8.000000,-1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000e+00,861.000000,7.000000,20.000000,0.000000e+00,8.000000,...,2014.00000,4.000000,2230.000000,0.000000,-1.000000,-1.000000,10.000000,2.000000,8.000000,-1.000000
75%,16.000000,0.000000,0.000000,0.000000,1.187425e+05,1428.000000,7.000000,46.000000,2.628630e+05,8.000000,...,3047.00000,4.000000,2984.250000,0.000000,-1.000000,1.000000,14.000000,2.000000,8.000000,0.000000
max,680.000000,5.000000,130668.000000,27.000000,6.048550e+06,131761.000000,140.000000,2300.000000,1.232559e+07,40.000000,...,4290.00000,6.000000,4290.000000,3.000000,0.000000,2.000000,22.000000,6.000000,29.000000,3.000000


# Divisão do conjunto de dados em treinamento, validação e verificação cruzada
Os dados originais serão divididos em tres conjuntos:

- Treinamento, com 90% dos dados (45.000 dados)
- Validação, com 5% dos dados (2.500 dados)
- Verificação cruzada, com 5% dos dados (2.500 dados)

In [22]:
# Fixa semente para reproduzir resultados
random.seed(42)

# Define treino como 90% dos dados de treino e
# validação como os 10% restantes, metade para testes e
# metade para validação cruzada
tamanhoTreino = math.floor(0.9 * numeroDados)

# Gera uma amostra aleatória com 90% dos índices
indicesTreino = random.sample(range(numeroDados), tamanhoTreino)
# Indices de Validação (testes) e Validação Cruzada
# Faz a diferença entre os conjuntos, garante que o resultado será ordenado
indicesValidacaoVC = sorted(list(set(range(numeroDados)) - set(indicesTreino)))
# e embaralha os índices antes de dividir entre validação e validação cruzada
random.shuffle(indicesValidacaoVC)
metade = len(indicesValidacaoVC)//2

indicesValidacao = indicesValidacaoVC[:metade]
indicesVCruzada = indicesValidacaoVC[metade:]

# Divide os dados nos tres subgrupos
# dadosTreino serão usados para treinar o modelo
# dadosValidacao serão usados para testar o modelo
# dadosVCruzada serão usados para estimar a performance do modelo
dadosTreino = kdd2009.iloc[indicesTreino].reset_index(drop=True)
dadosValidacao = kdd2009.iloc[indicesValidacao].reset_index(drop=True)
dadosVCruzada = kdd2009.iloc[indicesVCruzada].reset_index(drop=True)

# Dimensões dos dados de treino e validação
print("Dados de treinamento: {}, Dados de validação: {}, Dados de validação cruzada: {}".format(len(dadosTreino),
                                                                len(dadosValidacao),
                                                                len(dadosVCruzada)))

Dados de treinamento: 45000, Dados de validação: 2500, Dados de validação cruzada: 2500


## Verificando os dataframes

In [23]:
dadosTreino.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,392.0,0.0,NaN,NaN,NaN,...,-1,-1,-1,8,2,8,-1,False,False,False
1,NaN,NaN,NaN,NaN,NaN,1911.0,0.0,NaN,NaN,NaN,...,0,-1,-1,14,2,8,-1,False,False,False
2,NaN,NaN,NaN,NaN,NaN,854.0,7.0,NaN,NaN,NaN,...,0,-1,1,12,1,2,0,True,False,False
3,NaN,NaN,NaN,NaN,NaN,2331.0,7.0,NaN,NaN,NaN,...,0,-1,0,17,4,8,0,False,False,False
4,NaN,NaN,NaN,NaN,NaN,119.0,0.0,NaN,NaN,NaN,...,-1,-1,-1,1,0,8,-1,False,False,False


In [24]:
dadosValidacao.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,805.0,7.0,NaN,NaN,NaN,...,0,-1,-1,10,2,8,-1,False,False,True
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0,-1,-1,6,2,8,-1,False,False,False
2,NaN,NaN,NaN,NaN,NaN,1015.0,7.0,NaN,NaN,NaN,...,0,-1,0,0,0,25,0,False,False,False
3,NaN,NaN,NaN,NaN,NaN,294.0,0.0,NaN,NaN,NaN,...,0,-1,-1,5,2,8,-1,False,False,False
4,NaN,NaN,NaN,NaN,NaN,532.0,7.0,NaN,NaN,NaN,...,-1,-1,-1,18,1,13,-1,False,False,False


In [25]:
dadosVCruzada.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var223,Var224,Var225,Var226,Var227,Var228,Var229,appetency,churn,upselling
0,NaN,NaN,NaN,NaN,NaN,2737.0,0.0,NaN,NaN,NaN,...,0,-1,1,10,2,2,0,False,False,False
1,NaN,NaN,NaN,NaN,NaN,2527.0,7.0,NaN,NaN,NaN,...,0,-1,-1,14,2,8,-1,False,False,False
2,NaN,NaN,NaN,NaN,NaN,294.0,0.0,NaN,NaN,NaN,...,0,-1,-1,3,2,8,-1,False,False,False
3,NaN,NaN,NaN,NaN,NaN,1204.0,7.0,NaN,NaN,NaN,...,0,-1,0,22,3,26,-1,False,False,True
4,NaN,NaN,NaN,NaN,NaN,4312.0,7.0,NaN,NaN,NaN,...,3,-1,-1,16,3,25,1,False,True,False


## Verificando a distribuição dos rótulos entre treino, testes e validação cruzada

In [26]:
dadosTreino[resultados].sum()

appetency     805
churn        3341
upselling    3327
dtype: int64

In [27]:
dadosValidacao[resultados].sum()

appetency     41
churn        165
upselling    173
dtype: int64

In [28]:
dadosVCruzada[resultados].sum()

appetency     44
churn        166
upselling    182
dtype: int64

# XGBoost

Utilizaremos o [XGBoost](https://xgboost.readthedocs.io/en/latest/index.html) para treinamento dos dados do KDD2009 e predição de resultados no conjunto de testes.

De acordo com o a página de ajuda do XGBoost:
XGBoost é uma biblioteca otimizada de gradient boosting distribuído, projetada para ser altamente eficiente, flexível e portável. Implementa algoritmos de machine learning sob o framework de Gradient [Boosting](https://en.wikipedia.org/wiki/Boosting_(machine_learning)). O XGBoost provê uma árvore paralela de boosting (também conhecida como GBDT, GBM) que pode resolver vários problemas de ciência de dados de forma rápida e precisa. O mesmo código é executado em vários ambientes distribuídos (Hadoop, SCG, MPI) e pode resolver problemas com mais de bilhões de exemplos.


In [29]:
# Variável que será prevista
resultado = 'upselling'

## Rotinas de chamada do XGBoost
Para executar o XGBoost criamos duas rotinas:
- `gradeBoost`: Recebe dados de treinamento, de teste, o rótulo a ser testado e um iterador para a grade de treinamento, essa rotina "envelopa" a função `xgBoost`.
- `xgBoost`: Rotina que chama o XGBoost com os parâmetros desejados

A métrica utilizada para avaliar o modelo, de acordo com o desafio KDD2009 é o AUC:

**TODO**: Explicar AUC e ROC (que será incluído mais tarde)

In [30]:
def matrizTeste(teste, listaVariaveis):
    return xgb.DMatrix(teste[listaVariaveis])

def matrizTreino(treinamento, listaVariaveis):
    return xgb.DMatrix(treinamento[listaVariaveis], treinamento[[rotulo]])

def computaAUC(dados, listaVariaveis, rotulo, modelo):
    matriz = matrizTeste(dados, listaVariaveis)
    resultadoValidacao = modelo.predict(matriz)
    validacao = dados[rotulo].apply(int).reset_index(drop=True)
    previsao = 'previsao'
    dfPrevisao = pd.DataFrame(resultadoValidacao, columns=[previsao])
    dfPrevisao[resultado] = validacao
    dfPrevisao = dfPrevisao.sort_values(by=previsao)
    return auc(dfPrevisao[previsao], dfPrevisao[rotulo])

def gradeBoost(treinamento, listaVariaveis, teste, rotulo, intervaloGrade, verbose=0):
    """ Gera lista de resultados da grade de treinamento.
    Parâmetros:
    treinamento: Dataframe de dados de treinamento, com variaveis e rotulos
    listaVariaveis: Lista de variáveis para treinamento/teste
    teste: Dataframe de dados de teste/validação
    rotulo: Rotulo do teste
    intervaloGrade: Range de intervalo da grade de treinamento
    """
    # TODO: Tratamento de parâmetros/datarames de entrada
    
    # Converte dataframes para DMatrices
    xgbDados = matrizTreino(treinamento, listaVariaveis)
    xgbTeste = matrizTeste(teste, listaVariaveis)
    
    # Converte rotulos booleanos para inteiro (embora não seja estritamente necessário em python)
    validacao = teste[rotulo].apply(int).reset_index(drop=True)

    # Vetor de resultados
    resultados = []
    melhorAUC = float("-inf")
    melhorRodadas = -1
    dataFrame, modelo = (None, None)
    
    # Indices das tuplas
    rodadaXGB = 0
    aucRodada = -1
    for rodada in intervaloGrade:
        resultado = xgBoost(xgbDados, xgbTeste, validacao, rotulo, rodada, verbose=verbose)
        if resultado['resultado'][aucRodada] > melhorAUC:
            melhorAUC = resultado['resultado'][aucRodada]
            melhorRodadas = resultado['resultado'][rodadaXGB]
            dataFrame = resultado['df']
            modelo = resultado['modelo']
        resultados.append(resultado['resultado'])
            
    return {'melhor': {'auc': melhorAUC,
                       'rodadas': melhorRodadas,
                       'dataframe': dataFrame,
                       'modelo': modelo},
            'resultados': resultados}

def xgBoost(treinamento, teste, validacao, rotulo, rodadas, verbose=0):
    """ Executa o xgBoost e retorna tupla com número de rodadas e AUC
    """
    param = {'booster': 'dart',
             #'objective': 'binary:logistic',
             #'eta': 0.05,
             #'gamma': 0,
             'max_depth': 8,
             #'min_child_weight': 1,
             #'subsample': 1,
             #'colsample_bytree': 1
            }
    if verbose:
        print('XGBoost, {rodadas} rodadas'.format(rodadas=rodadas))
    # Treina modelo pelo número de rodadas determinado
    modelo = xgb.train(param, treinamento, rodadas)
    
    # Computa a previsão do modelo para o conjunto de teste
    resultadoValidacao = modelo.predict(teste)

    # Cria dataframe com previsão e rótulo
    previsao = 'previsao'
    dfPrevisao = pd.DataFrame(resultadoValidacao, columns=[previsao])
    dfPrevisao[rotulo] = validacao
    
    # Ordena o dataframe pela coluna de previsao
    dfPrevisao = dfPrevisao.sort_values(by=previsao)
    # Usamos o AUC do sklearn.metrics
    # requer: from sklearn.metrics import auc
    aucRun = auc(dfPrevisao[previsao], dfPrevisao[rotulo])
    if verbose:
        print('----------> AUC: {}'.format(aucRun))
    return {'df': dfPrevisao, 'modelo': modelo, 'resultado':(rodadas, aucRun)}
            

# Visualização de resultados
Rotinas bokeh para visualização de gráficos de linhas dos resultados

In [31]:
# Rotinas para geração de gráficos
def linha(dados, x="x", y="y", fig=None, delay=False, **kwargs):
    p = figure(width=900, height=600) if fig == None else fig
    p.y_range = Range1d(0,1)
    p.line(x, y, source=dados, **kwargs)
    if delay == False:
        show(p)
    return p
    
def linhaAUC(dados, fig=None, delay=False, **kwargs):
    return linha(dados, x="Rodadas", y="AUC", fig=fig, delay=delay, **kwargs)

In [32]:
# Iterador para a grade de busca
inicio, fim, passo = (5, 151, 1)
iteracoes = range(inicio, fim, passo)
# Plataforma utilizada...
plataforma = os.name

In [33]:
def trataResultados(tipo, dados):
    """Recebe como entrada:
    tipo: String com o tipo do teste
    dados: Dados que serão utilizados para o teste
    Executa o grid search do XGBoost se não existir arquivo de dados e salva dados, senão lê arquivo de dados
    Retorna: dataframe com resultados, AUC máximo, número de rodadas do AUC máximo
    """
    arquivoResultados = "resultado_{tipo}_{plataforma}_{inicio}-{fim}-{passo}.d".format(tipo=tipo,
                                                                                 plataforma=plataforma,       
                                                                                 inicio=inicio,
                                                                                 fim=fim,
                                                                                 passo=passo)
    arquivoModelo = "modelo_{tipo}_{plataforma}_{inicio}-{fim}-{passo}.d".format(tipo=tipo,
                                                                                 plataforma=plataforma,       
                                                                                 inicio=inicio,
                                                                                 fim=fim,
                                                                                 passo=passo)
    gradeResultados = os.path.join(".", "resultados", arquivoResultados)
    # Se os resultados já foram gerados para esta plataforma, lê estes resultados
    if os.path.isfile(gradeResultados):
        print("Lendo dados de {tipo}".format(tipo=tipo))
        resultadoGrade = pickle.load(open(gradeResultados, 'rb'))
    else: # Senão gera o dataframe correspondente e grava em disco
        print("Gerando arquivo de {tipo}".format(tipo=tipo))
        grade = gradeBoost(dadosTreino, variaveis, dados, resultado, iteracoes, verbose=1)
        aucMax = grade['melhor']['auc']
        numRodadas = grade['melhor']['rodadas']
        dfGrade = pd.DataFrame({"AUC":[g[1] for g in grade['resultados']],
                                "Rodadas":[g[0] for g in grade['resultados']]})
        resultadoGrade = {"dataframe": dfGrade, "melhor":grade["melhor"]}
        pickle.dump(resultadoGrade, open(gradeResultados, 'wb'))
        
    print("Fim geração/leitura de resultados")    
    return resultadoGrade

In [34]:
grade = trataResultados("validacao", dadosValidacao)
dfGrade = grade['dataframe']
aucMax = grade['melhor']['auc']
numRodadas = grade['melhor']['rodadas']
print('AUC Máximo: {}, {} rodadas'.format(aucMax, numRodadas))

Lendo dados de validacao
Fim geração/leitura de resultados
AUC Máximo: 0.6170178651809692, 142 rodadas


In [35]:
figTeste = linhaAUC(dfGrade, color="red", legend="Teste")

## Resultado da Validação Cruzada
A medida AUC estimada *realista* é dada pelo ***mínimo*** entre a melhor performance dos dados de **Teste** e o resultado da **Validação Cruzada**

In [36]:
aucVC = computaAUC(dadosVCruzada, variaveis, rotulo, grade['melhor']['modelo'])
print('Performance estimada do modelo (AUC): {}'.format(min(aucMax, aucVC)))

Performance estimada do modelo (AUC): 0.6170178651809692


# Comparação de comportamento entre Testes e Validação Cruzada

Para ter uma ideia mais clara da previsibilidade do modelo, vamos comparar os resultados globais dos dados de Testes (e não apenas o melhor resultado) com os resultados com os dados de Validação Cruzada 

In [37]:
gradeVCruzada = trataResultados("validacao_cruzada", dadosVCruzada)
dfGradeVCruzada = gradeVCruzada['dataframe']

Lendo dados de validacao_cruzada
Fim geração/leitura de resultados


In [38]:
figCombinada = linhaAUC(dfGradeVCruzada, fig=figTeste, legend="Validação Cruzada")

In [39]:
arquivoSaida = os.path.join('imagens', 'xgBoost_AUC_{rotulo}_{plataforma}.png'.format(rotulo=resultado,
                                                                                      plataforma=plataforma))
figCombinada.toolbar_location = None
figCombinada.toolbar.logo = None
export_png(figCombinada, filename=arquivoSaida)

'/Users/stelling/htdocs/KDD2009/python/imagens/xgBoost_AUC_upselling_posix.png'

[](file:///media/stelling/OS/xampp/htdocs/KDD2009/python/imagens/xgBoost_AUC_churn.png')

# AUC/Rodada

#### Ubuntu/MacOS (posix)
![nada](imagens/xgBoost_AUC_upselling_posix.png)

# Conclusão
**TODO**
